In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def _compute_errors(model_evaluation):
    model_evaluation[absolute_prediction_error] = model_evaluation[velocity_column] - model_evaluation[predictions_colum]
    model_evaluation[relative_prediction_error] = model_evaluation[absolute_prediction_error] / model_evaluation[velocity_column] * 100

In [ ]:
plot_scenario += 1

if plot_scenario == 1:
    model_folder = "DEFINE"
    gaps= []
    scenario = "Scenario 1"
    
if plot_scenario == 2:
    model_folder = "DEFINE"
    gaps = [
        (0.575, 0.625), # DEFINE, or REMOVE
        (0.65, 0.75) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 2"
    
if plot_scenario == 3:
    model_folder = "DEFINE"
    gaps = [
        (0,.62), # DEFINE, or REMOVE
        (0.73, 0.8048708335627524) # manually corrected from (0.73,1) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 3"

if plot_scenario == 4:
    model_folder = "DEFINE"
    gaps = [
        (.62,.73) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 4"
    
    
positions = ["pos3"]

absolute_prediction_error = "Absolute Prediction Error [m/s]"
relative_prediction_error = "Relative Prediction Error [%]"
velocity_column = "Velocity"
predictions_colum = "Predictions"

all_models = list()
model_performance = list()
best_model_mean = None
best_model_range = None
best_absolute_mean = None

best_absolute_quantile_range = None

evaluation_key = "_evaluation_set_result.csv"

for current_experiment in positions:
    
    for file in os.listdir(model_folder):

        if not current_experiment in file:
            continue

        if evaluation_key in file:

            model_name_end_index = file.index(evaluation_key)

            model_name = file[0:model_name_end_index]
            all_models.append(model_name)

    for model in all_models:
        
        if not "m_4_conv_5" in model:
            continue

        evaluation_path = model_folder + model + evaluation_key
        evaluation = pd.read_csv(evaluation_path)

        _compute_errors(evaluation)

        mean = evaluation[relative_prediction_error].pow(2).apply(np.sqrt).mean()
        model_performance.append((mean, model))
        
        absolute_mean = abs(mean)

        quantiles = evaluation[relative_prediction_error].quantile([.05, .95])

        quantile_range = quantiles[0.95] - quantiles[0.05]

        print("%s mean error: %.2f" % (model, mean))
        print("%s quantile range: %.2f\n" % (model, quantile_range))

        if best_absolute_mean is None or absolute_mean < best_absolute_mean:
            best_absolute_mean = absolute_mean
            best_model_mean = model

        if best_absolute_quantile_range is None or quantile_range < best_absolute_quantile_range:
            best_absolute_quantile_range = quantile_range
            best_model_range = model

    print("Best model (mean) is: %s with mean error: %.2f" % (best_model_mean, best_absolute_mean))
    print("Best model (quantile range) is: %s with quantile range: %.2f" % (best_model_range, best_absolute_quantile_range))

model_name = best_model_range
model_name = best_model_mean

evaluation_path = model_folder + model_name + evaluation_key
model_name

In [ ]:
performance = np.array(model_performance)
performance = pd.DataFrame({"Name" : performance[:,1], "Error": performance[:,0]})
performance = performance.astype({"Error" : "float"})
tmp_sorted = performance.sort_values(by="Error")
tmp_sorted

In [ ]:
evaluation_complete = pd.read_csv(evaluation_path)
_compute_errors(evaluation_complete)

evaluation_complete = evaluation_complete.sort_values(by="Frame", ignore_index=True)

evaluation_complete

In [ ]:
evaluation = pd.read_csv(evaluation_path)
_compute_errors(evaluation)

evaluation = evaluation[[velocity_column, predictions_colum, absolute_prediction_error, relative_prediction_error]]

evaluation

In [ ]:
pd.__version__

In [ ]:
lower_quantile = .05
upper_quantile = .95

In [ ]:
absolute_error_quantiles = evaluation[absolute_prediction_error].quantile([lower_quantile, upper_quantile])
absolute_error_quantiles

In [ ]:
relative_error_quantiles = evaluation[relative_prediction_error].quantile([lower_quantile, upper_quantile])
relative_error_quantiles

In [ ]:
rrmse = evaluation[relative_prediction_error].pow(2).apply(np.sqrt).mean()

In [ ]:
rolling_window_size = 50

figure, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation[velocity_column].plot(ax=axes[0], label="Measurements", title="Measurements vs. Predictions")
evaluation[predictions_colum].plot(ax=axes[0])

axes[0].legend()

evaluation[absolute_prediction_error].plot(ax=axes[1], title=absolute_prediction_error)
evaluation[absolute_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=absolute_prediction_error)

axes[1].axhline(evaluation[absolute_prediction_error].mean(), color="green")
axes[1].axhline(absolute_error_quantiles[lower_quantile], color="red")
axes[1].axhline(absolute_error_quantiles[upper_quantile], color="red")

axes[1].axhline(min(evaluation[absolute_prediction_error]), color="black")
axes[1].axhline(max(evaluation[absolute_prediction_error]), color="black")

evaluation[relative_prediction_error].plot(ax=axes[2], title=relative_prediction_error)
evaluation[relative_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[2], title=relative_prediction_error)

axes[2].axhline(evaluation[relative_prediction_error].mean(), color="green")
axes[2].axhline(relative_error_quantiles[lower_quantile], color="red")
axes[2].axhline(relative_error_quantiles[upper_quantile], color="red")

axes[2].axhline(min(evaluation[relative_prediction_error]), color="black")
axes[2].axhline(max(evaluation[relative_prediction_error]), color="black")

for axis in axes:
    for gap in gaps:
        gap_lower = np.argmax(evaluation[velocity_column] >= gap[0])
        gap_upper = np.argmax(evaluation[velocity_column] >= gap[1])
        axis.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

plt.show()
plt.close()

In [ ]:
rolling_window_size = 50

figure, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation[velocity_column].plot(ax=axes[0], label="Measurements", title="Measurements vs. Predictions", fontsize="x-large")
evaluation[predictions_colum].plot(ax=axes[0])

axes[0].legend()
axes[0].set_ylabel("Surface Velocity [m/s]", fontsize="x-large")

evaluation[absolute_prediction_error].plot(ax=axes[1], title=absolute_prediction_error)
evaluation[absolute_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=absolute_prediction_error)

evaluation[relative_prediction_error].plot(ax=axes[1], title=relative_prediction_error)
evaluation[relative_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=relative_prediction_error, fontsize="x-large")

axes[1].axhline(evaluation[relative_prediction_error].mean(), color="green")
axes[1].axhline(relative_error_quantiles[lower_quantile], color="red")
axes[1].axhline(relative_error_quantiles[upper_quantile], color="red")

axes[1].axhline(min(evaluation[relative_prediction_error]), color="black")
axes[1].axhline(max(evaluation[relative_prediction_error]), color="black")

axes[1].set_xlabel("Measurement", fontsize="x-large")
axes[1].set_ylabel("Error [%]", fontsize="x-large")

plt.subplots_adjust(hspace = .2)
plt.show()
plt.close()

In [ ]:
rolling_window_size = 10

figure, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation[predictions_colum].plot(ax=axes, color="#abd9e9", alpha=.6)
evaluation[predictions_colum].rolling(rolling_window_size).mean().plot(ax=axes, color="#2c7bb6")
evaluation[velocity_column].plot(ax=axes, label="Measurements", color="#d7191c", fontsize="x-large")


axes.legend(["Predictions", "Predictions (Rolling Average)", "Measurements"])
axes.set_ylabel("Surface Velocity [m/s]", fontsize="x-large")
axes.set_xlabel("# Measurements", fontsize="x-large")
    
for axis in [axes]:
    for gap in gaps:
        gap_lower = np.argmax(evaluation[velocity_column] >= gap[0])
        gap_upper = np.argmax(evaluation[velocity_column] >= gap[1])
        axis.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

plt.show()
plt.close()

In [ ]:
print("Mean relative prediction error: %.2f" % evaluation[relative_prediction_error].mean())
print("Mean relative prediction error 5th percentile: %.2f%%" % relative_error_quantiles[lower_quantile])
print("Mean relative prediction error 95th percentile: %.2f%%" % relative_error_quantiles[upper_quantile])

In [ ]:
number_of_bins = 21 # 5% steps
bin_percentages = np.linspace(0,100,21)
bin_labels = ["L%d" % i for i in bin_percentages]
percentage_label = "Percentage Label"

labels, bins = pd.cut(evaluation[velocity_column], number_of_bins, labels=bin_labels, retbins=True)
evaluation[percentage_label] = labels

group_means = evaluation.groupby(by=percentage_label).mean()
group_quantiles = evaluation[[percentage_label, relative_prediction_error]].groupby(by=percentage_label).quantile([.05, .95])

group_quantiles.index.set_names(["Label", "Quantile"], inplace=True)

upper_quantiles = group_quantiles.iloc[group_quantiles.index.get_level_values('Quantile') == upper_quantile]
lower_quantiles = group_quantiles.iloc[group_quantiles.index.get_level_values('Quantile') == lower_quantile]

lower = lower_quantiles[relative_prediction_error].values
upper = upper_quantiles[relative_prediction_error].values
means = group_means[relative_prediction_error].values


In [ ]:
fig = plt.figure(figsize=(10, 10/1.6))
fig.tight_layout()

zero_line_y = np.zeros(len(bin_labels))

plt.plot(bin_labels, lower)
plt.plot(bin_labels, upper)
plt.plot(bin_labels, means)

plt.plot(bin_labels, zero_line_y, alpha=0)
plt.fill_between(bin_labels, lower, upper, facecolor="C0", alpha=0.05)
plt.fill_between(bin_labels, zero_line_y, means, where=(zero_line_y < means), facecolor="C1", alpha=0.35)
plt.fill_between(bin_labels, zero_line_y, means, where=(zero_line_y > means), facecolor="C2", alpha=0.35)


plt.show()
plt.close()

In [ ]:
factor = 10

new_bin_indexes = np.linspace(0, 100, number_of_bins * factor)

lower_inderpolated = np.interp(new_bin_indexes, bin_percentages, lower)
upper_inderpolated = np.interp(new_bin_indexes, bin_percentages, upper)
means_inderpolated = np.interp(new_bin_indexes, bin_percentages, means)

In [ ]:
fig = plt.figure(figsize=(10, 10/1.6))
fig.tight_layout()

zero_line_y = np.zeros(len(lower_inderpolated))

plt.plot(new_bin_indexes, lower_inderpolated)
plt.plot(new_bin_indexes, upper_inderpolated)
plt.plot(new_bin_indexes, means_inderpolated)

plt.ylabel("Relative Error (Expected-Prediction) [%%]")
plt.xlabel("Percentage Full [%]")

plt.plot(new_bin_indexes, zero_line_y, alpha=0)
plt.fill_between(new_bin_indexes, lower_inderpolated, upper_inderpolated, facecolor="C0", alpha=0.05)
plt.fill_between(new_bin_indexes, zero_line_y, means_inderpolated, where=(zero_line_y < means_inderpolated), facecolor="C1", alpha=0.35)
plt.fill_between(new_bin_indexes, zero_line_y, means_inderpolated, where=(zero_line_y > means_inderpolated), facecolor="C2", alpha=0.35)

plt.xticks((bin_percentages))

plt.show()
plt.close()

In [ ]:
table_entry_unsorted = {
    "Title" : scenario,
    "Name" : model_name,
    "RRMSE" : rrmse,
    "Percentiles" : relative_error_quantiles,
    "Measurements" : evaluation[velocity_column],
    "Predictions" : evaluation[predictions_colum],
    "Gaps" : gaps
}

In [ ]:
table_contents_unsorted.append(table_entry_unsorted)

In [ ]:
evaluation = evaluation.sort_values(by=velocity_column, ignore_index=True)

In [ ]:
table_entry = {
    "Title" : scenario,
    "Name" : model_name,
    "RRMSE" : rrmse,
    "Percentiles" : relative_error_quantiles,
    "Measurements" : evaluation[velocity_column],
    "Predictions" : evaluation[predictions_colum],
    "Gaps" : gaps
}

In [ ]:
table_contents.append(table_entry)

In [ ]:
for t in table_contents:
    print(t["Name"])

In [ ]:
if (len(table_contents) > 3):
    backup = table_contents[2]["Gaps"]
    backup

In [ ]:
if (len(table_contents) > 3):
    table_contents[2]["Gaps"] = [(0, 0.62), (0.73, max(evaluation[velocity_column]))]

In [ ]:
table_contents[0]["Title"] = "Scenario 1 %s"
table_contents[1]["Title"] = "Scenario 2 %s"
table_contents[2]["Title"] = "Scenario 3 %s"
table_contents[3]["Title"] = "Scenario 4 %s"

In [ ]:
rolling_window_size = 10

n_rows = 2
n_cols = int(len(table_contents) / n_rows)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=False, sharey=True, figsize=(15,15),constrained_layout=True)

table_content_index = 0

for i in range(n_rows):
    for j in range(n_cols):
        
        table_content = table_contents[table_content_index]
        error_string = "\n[%.2f, %.2f, %.2f]" % (table_content["Percentiles"][0.05], table_content["RRMSE"], table_content["Percentiles"][0.95])

        table_content["Predictions"].plot(ax=axes[i][j], color="#abd9e9", alpha=.6)
        table_content["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes[i][j], color="#2c7bb6")
        table_content["Measurements"].plot(ax=axes[i][j], label="Measurements", color="#fdae61")

        axes[i][j].legend(["Predictions", "Predictions (Rolling Average)", "Measurements"],loc='upper left', prop={'size': 18})
        axes[i][j].set_ylabel("Surface Velocity [m/s]", fontsize="xx-large")
        axes[i][j].set_xlabel("Measurements", fontsize="x-large")
#         axes[i][j].set_xlabel("Measurements\n%s" % error_string, fontsize="x-large")
        axes[i][j].set_title(table_content["Title"] % error_string, fontsize="x-large")
    
        axes[i][j].tick_params(axis='both', which='major', labelsize="x-large")
        axes[i][j].tick_params(axis='both', which='minor', labelsize="x-large")
    
        #axes[ax_index].grid()
        

        for gap in table_content["Gaps"]:
            gap_lower = np.argmax(evaluation[velocity_column] >= gap[0])
            gap_upper = np.argmax(evaluation[velocity_column] >= gap[1])
        
            axes[i][j].axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

        table_content_index += 1

figure.suptitle(u"Surface Velocity Prediction Scenario Evaluation", fontsize="20")


plt.show()
plt.close()

In [ ]:
rolling_window_size = 10

n_rows = 2
n_cols = int(len(table_contents) / n_rows)


figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=False, sharey=True, figsize=(15,15),constrained_layout=True)

table_content_index = 0

gap_color = "#e66101"
predictions_color = "#b2abd2"
rolling_average_color = "#5e3c99"
measurements_color = "#fdb863"

for i in range(n_rows):
    for j in range(n_cols):
            
        table_content = table_contents[table_content_index]
        
        table_content["Predictions"].plot(ax=axes[i][j], color=predictions_color, alpha=.6)
        table_content["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes[i][j], color=rolling_average_color)
        table_content["Measurements"].plot(ax=axes[i][j], label="Measurements", color=measurements_color)

        if i == 0 and j == 0:
            axes[i][j].legend(["Predictions", "Predictions (Rolling Average)", "Measurements"],loc='upper left', prop={'size': 18})
        axes[i][j].set_ylabel("Surface Velocity [m/s]", fontsize="xx-large")
        axes[i][j].set_xlabel("Measurements", fontsize="x-large")
        axes[i][j].set_title(table_content["Title"] % "", fontsize="xx-large")
        
        axes[i][j].tick_params(axis='both', which='major', labelsize="x-large")
        axes[i][j].tick_params(axis='both', which='minor', labelsize="x-large")
        
        axes[i][j].set_xlim(0,len(table_content["Measurements"]))
        axes[i][j].set_ylim(table_content["Measurements"].min(),table_content["Measurements"].max())
        

        for gap in table_content["Gaps"]:
            gap_lower = np.argmax(evaluation[velocity_column] >= gap[0])
            gap_upper = np.argmax(evaluation[velocity_column] >= gap[1])
            axes[i][j].axvspan(gap_lower, gap_upper, alpha=.1, color=gap_color)
        
        error_string = "Error Summary: [%.2f, %.2f, %.2f]" % (table_content["Percentiles"][0.05], table_content["RRMSE"], table_content["Percentiles"][0.95])
        error_string = \
            "Prediction Errors:\n" + \
            "5$^{th}$ percentile:\t%.2f\n" % table_content["Percentiles"][0.05] + \
            "95$^{th}$ percentile: \t%.2f\n" % table_content["Percentiles"][0.95] + \
            "RRMSE:\t\t%.2f" % table_content["RRMSE"]
        
        error_string =  f"{'RRMSE:':<25}|{table_content['RRMSE']:>10.2f}\n"  +\
                        f"{'5$^{th}$ percentile:':<25}|{table_content['Percentiles'][0.05]:>10.2f}\n" +\
                        f"{'95$^{th}$ percentile:':<25}|{table_content['Percentiles'][0.95]:>10.2f}"
        
        error_string =  f"{table_content['RRMSE']:>10.2f} {'RRMSE':>25}\n"  +\
                        f"{table_content['Percentiles'][0.05]:>10.2f}{'5$^{th}$ percentile':>30}\n" +\
                        f"{table_content['Percentiles'][0.95]:>10.2f}{'95$^{th}$ percentile':>29}"
            
        
        axes[i][j].text(200,.5,error_string, fontsize = "xx-large", linespacing=1.1)

        table_content_index += 1



plt.show()
plt.close()

In [ ]:
table_content = table_contents_unsorted[0]

table_content["Predictions"] = pd.DataFrame(table_content["Predictions"])
table_content["Measurements"] = pd.DataFrame(table_content["Measurements"])

In [ ]:
print(table_content["Title"] )
print(float(table_content["Measurements"].min()))
print(float(table_content["Measurements"].max()))

n_rows = 1
n_cols = 1

rolling_window_size = 10

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=False, sharey=True, figsize=(10,10),constrained_layout=True)

gap_color = "#e66101"
predictions_color = "#b2abd2"
rolling_average_color = "#5e3c99"
measurements_color = "#fdb863"

predictions = table_content["Predictions"] 
rolling_predictions = table_content["Predictions"].rolling(rolling_window_size).mean() 
measurements = table_content["Measurements"].rolling(rolling_window_size).mean() 
        
rolling_predictions.plot(ax=axes, color=rolling_average_color)
measurements.plot(ax=axes, label="Measurements", color=measurements_color)

axes.legend(["Predictions", "Predictions (Rolling Average)", "Measurements"],loc='upper right', prop={'size': 18})
axes.set_ylabel("Surface Velocity [m/s]", fontsize="xx-large")
axes.set_xlabel("Time[s]", fontsize="x-large")
axes.set_title(table_content["Title"], fontsize="xx-large")
        
axes.tick_params(axis='both', which='major', labelsize="x-large")
axes.tick_params(axis='both', which='minor', labelsize="x-large")
        
max_x = table_content["Measurements"].index.max()
axes.set_xlim(0, table_content["Measurements"].index.max())
axes.set_ylim(float(table_content["Measurements"].min()), float(table_content["Measurements"].max()))

error_string =  f"{table_content['RRMSE']:>10.2f} {'RRMSE':>25}\n"  +\
                f"{table_content['Percentiles'][0.05]:>10.2f}{'5$^{th}$ percentile':>30}\n" +\
                f"{table_content['Percentiles'][0.95]:>10.2f}{'95$^{th}$ percentile':>29}"
            
        
axes.text(200,.5,error_string, fontsize = "xx-large", linespacing=1.1)

plt.show()
plt.close()



In [ ]:
evaluation

In [ ]:
table_contents

In [ ]:
len(table_contents)

In [ ]:
table_contents[0]["Title"] = "Scenario 1 %s"
table_contents[1]["Title"] = "Scenario 2 %s"
table_contents[2]["Title"] = "Scenario 3 %s"
table_contents[3]["Title"] = "Scenario 4 %s"

In [ ]:
execution_blocker

In [ ]:
table_contents = list()
table_contents_unsorted = list()
plot_scenario = 0